# Text Generation

* 게임플랫폼 '스팀'의 리뷰 생성기를 제작해보자.

# Data : Steam reviews


In [120]:
# 처음 설치할때만 실행하고, 그 후에는 주석처리 하세요.
!pip install konlpy

In [121]:
import numpy as np
import pandas as pd

In [122]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/steam.txt'
data= pd.read_table(url, names=['label', 'reviews'])

# Q1. data안의 'reviews'만 가져와 전처리 하시오.

1. 'reviews'컬럼만 사용한다.
2. 'reviews'의 문장들을 하나의 list로 담는다.
3. reviews에 선언한다.
4. 처음 10000개 문장만 사용하기로 한다.(그렇지 않으면 매우 느려진다.)

In [123]:
####################
## Your Code here ##
####################

reviews = data['reviews'].tolist()[:10000]

# (+optional)Q2. reviews안의 모든 문장에, 형태소 사이 사이에 띄어쓰기를 추가하라.

* 없이 진행해보고, 나중에 추가하여 비교해보자.
* konlpy의 Okt를 사용한다.
* 시간이 꽤 걸릴 수 있으니, 처음에는 정상적으로 루프가 동작하는지 처음 10개 리뷰만 가지고 테스팅해가며 진행한다.
* 전체 문장을 전부 진행할경우, 약 1~2분 정도 걸린다.

In [124]:
# %%time

# ####################
# ## Your Code here ##
# ####################
# from konlpy.tag import Okt
# okt = Okt()


# print("변경 전 :", reviews[5])

# for idx, sentence in enumerate(reviews) :
#     reviews[idx] = " ".join( okt.morphs(sentence)  )

# print("변경 후 :", reviews[5])


# Q3. reviews를 띄어쓰기 기준으로 tokenize하고, index의 sequence로 바꾸어 내시오.

**조건**
* revews_seq 에 선언해둔다.
* 패딩에 사용할 0을 포함하여, 상위 20000개 단어를 사용하자.
    * tokenizer는 num_words에 20000을 주면, 사용자가 0을 사용할 것을 감안하여 상위 1999개만 추려준다.  패딩 포함하면 자동으로 20000개가 된다.
    * max_words = 20000 을 활용한다.

In [125]:
####################
## Your Code here ##
####################
from tensorflow.keras.preprocessing.text import Tokenizer

max_words = 20000

tk = Tokenizer(num_words=max_words)
tk.fit_on_texts(reviews)
reviews_seq = tk.texts_to_sequences(reviews)

# Q4.reviews_seq 에서 길이가 2이하인 seq를 제거하시오.

* 그대로 다시 reviews_seq에 선언한다.

In [126]:
reviews_seq = [ seq for seq in reviews_seq if len(seq)>=3 ]

reviews_seq[:6]

[[1229, 2, 1762],
 [10391, 13, 10392, 2472, 5838, 1763, 10393, 1764],
 [5839, 5840, 5839, 10394, 3075, 21, 10395, 766],
 [4007, 4007, 4007, 11, 4008, 5841, 10396, 7, 5842],
 [133, 10397, 139, 1116, 10398, 10, 1765, 7, 344],
 [95, 826, 2473, 498, 2474, 3076, 5843, 10399, 452]]

# Q5. reviews_seq안의 모든 문장을 다음과 같이 재구성하시오.

**예시**
* reviews_seq가 [ [1,2,3], [7,2,9,10] ] 라면
* 결과 : reviews_seq = [ [1, 2], [1,2,3], [7,2], [7,2,9], [7,2,9,10] ]
* 중간에 다른 변수를 임시로 사용해도 좋다.

In [127]:
####################
## Your Code here ##
####################

temp_ = []

for seq in reviews_seq :
    for i in range(1, len(seq)):
        temp_.append( seq[:i+1]    )

reviews_seq = temp_
reviews_seq[:10]


[[1229, 2],
 [1229, 2, 1762],
 [10391, 13],
 [10391, 13, 10392],
 [10391, 13, 10392, 2472],
 [10391, 13, 10392, 2472, 5838],
 [10391, 13, 10392, 2472, 5838, 1763],
 [10391, 13, 10392, 2472, 5838, 1763, 10393],
 [10391, 13, 10392, 2472, 5838, 1763, 10393, 1764],
 [5839, 5840]]

# Q6. reviews_seq안의 seq의 길이를 통일하시오.

**Questions**
1. 0을 앞에 덧대야 할까, 뒤에 덧대야 할까?
2. max_len = 11 ( x로 활용할 10, y로 활용할 1 )
3. reviews_seq의 shape를 확인하라.

In [128]:
####################
## Your Code here ##
####################


from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 10 + 1

reviews_seq = pad_sequences(reviews_seq, maxlen=max_len, padding='pre')

In [129]:
reviews_seq.shape

(73484, 11)

# Q7. reviews_seq의 마지막 컬럼은 y로, 나머지 컬럼은 x로 분할하라.

In [130]:
####################
## Your Code here ##
####################

x = reviews_seq[:, :-1]
y = reviews_seq[:, -1]


# 주의. y를 원핫 인코딩 하지 않을 예정. 메모리가 버티지 못할 수 있다.

# Q8. 문장을 받아, 다음 단어를 선택하는 모델을 만드시오.

* 컴파일까지 마치시오.
    * 모니터링 지표로 accuracy 사용
    * loss 를 sparse_categorical_crossentropy를 사용해야만, y를 원핫인코딩 하지 않고도 학습시킬 수 있다.
* 모델 구조는 아래와 같다.
    1. 임베딩 레이어 : 임베딩 차원 256, 문장길이 10
    2. GRU, 128, 모든 출력을 다음 layer로
    3. GRU, 128, 모든 출력을 다음 layer로
    4. MaxPool1D, 필터사이즈 2
    4. GRU, 256, 모든 출력만 다음 레이어로.
    4. GRU, 256, 모든 출력만 다음 레이어로.
    4. MaxPool1D, 필터사이즈 2
    4. GRU, 512, 마지막 출력만 다음 레이어로.
    5. 멀티클래스(max_words만큼)분류하는 아웃풋레이어


In [131]:
####################
## Your Code here ##
####################
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import GRU, Embedding, Dense, MaxPool1D

In [132]:
####################
## Your Code here ##
####################

keras.backend.clear_session()

model = keras.models.Sequential()

model.add( keras.layers.Embedding(input_dim = max_words,
                                  output_dim = 256,
                                  input_length = 10 ))
model.add( GRU(128, return_sequences=True) )
model.add( GRU(128, return_sequences=True) )
model.add( MaxPool1D( 2 ) )
model.add( GRU(256, return_sequences=True) )
model.add( GRU(256, return_sequences=True) )
model.add( MaxPool1D( 2 ) )
model.add( GRU(512, return_sequences=False) )
model.add( Dense( max_words, activation='softmax' ))

model.compile( loss = 'sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 256)           5120000   
_________________________________________________________________
gru (GRU)                    (None, 10, 128)           148224    
_________________________________________________________________
gru_1 (GRU)                  (None, 10, 128)           99072     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 5, 128)            0         
_________________________________________________________________
gru_2 (GRU)                  (None, 5, 256)            296448    
_________________________________________________________________
gru_3 (GRU)                  (None, 5, 256)            394752    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2, 256)            0

# Q8. 모델을 학습시킬 것.

* batch_size = 1024
* epochs = 60


In [133]:
####################
## Your Code here ##
####################

model.fit(x, y, epochs=60, batch_size=1024, verbose=1)

Epoch 1/60
72/72 [==============================] - 21s 114ms/step - loss: 9.2779 - accuracy: 0.0173
Epoch 2/60
72/72 [==============================] - 8s 112ms/step - loss: 8.7870 - accuracy: 0.0178
Epoch 3/60
72/72 [==============================] - 8s 112ms/step - loss: 8.7210 - accuracy: 0.0178
Epoch 4/60
72/72 [==============================] - 8s 112ms/step - loss: 8.6777 - accuracy: 0.0178
Epoch 5/60
72/72 [==============================] - 8s 113ms/step - loss: 8.6114 - accuracy: 0.0177
Epoch 6/60
72/72 [==============================] - 8s 114ms/step - loss: 8.5335 - accuracy: 0.0178
Epoch 7/60
72/72 [==============================] - 8s 114ms/step - loss: 8.4659 - accuracy: 0.0179
Epoch 8/60
72/72 [==============================] - 8s 113ms/step - loss: 8.3928 - accuracy: 0.0180
Epoch 9/60
72/72 [==============================] - 8s 113ms/step - loss: 8.3020 - accuracy: 0.0182
Epoch 10/60
72/72 [==============================] - 8s 113ms/step - loss: 8.1960 - accuracy: 0.018

# (+권장)Q9. 다음의 함수를 제작한다.

* 인풋 : 사용할 첫 단어, 반복 예측 수, 사용할 모델, 사용할 tokenizer
* 내부 : 모든 예측 중 하나를 확률에 맞추어 sampling 한다.
    * np.random.choice 이용. 만일 '바람'일 확률이 80%라면, 100번에 80번 정도는 바람이, 20번 정도는 다른 단어가 선택될 것이다.
* 아웃풋 : 완성된 문장.
* 반복 예측 수 : 9이상이면 다음의 문구를 출력하고 함수 동작을 종료한다.
    * "8이하의 반복예측만 가능합니다."
* 예시 :
```
reviewer("나", 4, tk, model )
>> "나 추억 한다면 힘차게 걸으리라"
reviewer("나", 4, tk, model )
>> "나 사랑 했다면 미소 를"
```

In [134]:
####################
## Your Code here ##
####################

def reviewer( word, n_iter, tk, model):
    if n_iter >= 9 :
        print("6이하의 반복 예측만 가능합니다.")
    else : 
        for i in range( n_iter ) :
            words = [word]
            seq_ = tk.texts_to_sequences(words)
            seq_ = pad_sequences(seq_, maxlen=max_len-1, padding='pre')
            pred_ = model.predict(seq_, )
            selected_idx = np.random.choice(max_words, p = pred_.reshape(-1))
            next_word = tk.index_word[ selected_idx ]

            word = word + " " + next_word
        return word

In [138]:
#######################
## Your testing here ##
#######################
reviewer("나", 7, tk, model)

'나 1 포기 게임 2판 사셈 시뮬레이팅 그만큼'